<a href="https://colab.research.google.com/github/DatacationOrg/chatbot-workshop/blob/main/Datacation_workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%writefile requirements.txt
chainlit==2.2.1
langchain==0.3.19
langchain-community==0.3.18
langchain-google-genai==2.0.11

In [ ]:
!pip install -r requirements.txt

In [ ]:
!npm install -g localtunnel

In [ ]:
%%writefile app.py
import os

import chainlit as cl
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-exp")


@cl.on_message
async def on_message(message: cl.Message):
    msg = cl.Message(content="")
    async for chunk in llm.astream(
        message.content, config=dict(callbacks=[cl.LangchainCallbackHandler()])
    ):
        await msg.stream_token(chunk.content)
    await msg.send()


In [ ]:
import os
import subprocess

from google.colab import userdata

chainlit_process = None
if chainlit_process:
    chainlit_process.kill()
tunnel_process = None
if tunnel_process:
    tunnel_process.kill()
chainlit_out = open("chainlit_output.log", "wb")
chainlit_env = os.environ.copy()
chainlit_env["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")
chainlit_process = subprocess.Popen(
    ["chainlit", "run", "app.py", "--host", "0.0.0.0", "--port", "8000"],
    stdout=chainlit_out,
    stderr=subprocess.STDOUT,
    env=dict(os.environ, GOOGLE_API_KEY=userdata.get("GOOGLE_API_KEY")),
)
output_file = "output.log"
out_log = open(output_file, "wb")
tunnel_process = subprocess.Popen(
    ["lt", "--port", "8000"], stdout=out_log, stderr=subprocess.STDOUT
)
!sleep 5s # Allow for some time to properly boot

In [ ]:
assert chainlit_process.poll() is None
assert tunnel_process.poll() is None
!cat $output_file
!echo "Tunnel Password: $(curl -s https://loca.lt/mytunnelpassword)"